In [ ]:
# Import Library
import shutil 
import tensorflow as tf
import keras
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
from tqdm import tqdm

from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.preprocessing import image

from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

drive.mount('/content/gdrive')

In [ ]:
# timer 
class TimerError(Exception):
  """A custom exception used to report errors in use of Timer class"""
class Timer:
  def __init__(self):
    self._start_time = None
  def start(self):
    """Start a new timer"""
    if self._start_time is not None:
      raise TimerError(f"Timer is running. Use .stop() to stop it")

    self._start_time = time.perf_counter()

  def stop(self):
    """Stop the timer, and report the elapsed time"""
    if self._start_time is None:
      raise TimerError(f"Timer is not running. Use .start() to start it")

    elapsed_time = time.perf_counter() - self._start_time
    self._start_time = None
    print(f"Executed time: {elapsed_time:0.4f} seconds")

In [ ]:
# load images and datset csv
image_directory = 'gdrive/My Drive/Dataset Skripsi/DS4 - ALL/'
df = pd.read_csv('gdrive/My Drive/Dataset Skripsi/DS4_CSV.csv')  

test_directory = 'gdrive/My Drive/Dataset Skripsi/Dataset Testing - Resized - All/'  
df_test = pd.read_csv('gdrive/My Drive/Dataset Skripsi/DATASET_TESTING.csv') 

In [ ]:
# preprocessing images
SIZE = 200
X_dataset = []  
for i in tqdm(range(df.shape[0])):
    img = image.load_img(image_directory +df['Name'][i]+'.jpg', target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset.append(img)
    
X = np.array(X_dataset)                     # array images
y = np.array(df.drop(['Name'], axis=1))     # array output class label

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)     # split for training and validation

In [ ]:
# define model cnn
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(SIZE,SIZE,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=300, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))

In [ ]:
# define model cnn 2
weight = 1
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(SIZE,SIZE,3), kernel_initializer=keras.initializers.GlorotUniform(seed=1231)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu',kernel_initializer=keras.initializers.GlorotUniform(seed=1231)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu",kernel_initializer=keras.initializers.GlorotUniform(seed=1231)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu',kernel_initializer=keras.initializers.GlorotUniform(seed=1231)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=300, kernel_size=(5, 5), activation='relu',kernel_initializer=keras.initializers.GlorotUniform(seed=1231)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_initializer=keras.initializers.GlorotUniform(seed=66)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu',kernel_initializer=keras.initializers.GlorotUniform(seed=66)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu',kernel_initializer=keras.initializers.GlorotUniform(seed=66)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid',kernel_initializer=keras.initializers.GlorotUniform(seed=66)))

In [ ]:
model.summary()

In [ ]:
display_activation(activations, 4, 4, 17)

In [ ]:
### step decay learning rate ###
initial_learning_rate = 0.0001
def lr_step_decay(epoch, lr):
    drop_rate = 0.3
    epochs_drop = 10.0
    return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))

In [ ]:
### time base decay learning rate
initial_learning_rate = 0.0001
epochs = 100
decay = initial_learning_rate / epochs
def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)

In [ ]:
### exponential decay learning rate 
initial_learning_rate = 0.01
def lr_exp_decay(epoch, lr):
    k = 0.1
    return initial_learning_rate * math.exp(-k*epoch)

In [ ]:
pat = 5 #this is the number of epochs with no improvment after which the training will stop
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

#define the model checkpoint callback -> this will keep on saving the model as a physical file
model_checkpoint = ModelCheckpoint('model_16_2.h5', verbose=1, save_best_only=True)

In [ ]:
# laod tensorboard
%load_ext tensorboard
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
#@title Default title text
#model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
#              loss='binary_crossentropy', 
#             metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy" ,threshold=0.8)])

#history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=30, 
#                    callbacks=[model_checkpoint])

#################### use early stoping, learning rate scheduler ##################################

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy" ,threshold=0.8)])

#model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

t = Timer()
t.start()
#history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=90, 
#                    callbacks=[keras.callbacks.LearningRateScheduler(lr_time_based_decay, verbose=1)])
#history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test), batch_size=100, callbacks=[early_stopping, model_checkpoint])
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=90, callbacks=[model_checkpoint])
t.stop()

#history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=90, callbacks=[model_checkpoint])

#history = model.fit(X, y, epochs=100, batch_size=50)

In [ ]:
# predict the testing dataset
predictions = model1.predict(XTest, verbose=1)

In [ ]:
# save prediction to csv
prediction = pd.DataFrame(predictions).to_csv('prediction_val_100.csv')

In [ ]:
# evaluate overall testing prediction 
model.evaluate(XTest, YTest, verbose=1)

In [ ]:
# save history training
hist_df = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'hist_acc_final_2.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# copy to google drive
shutil.copy('hist_acc_final_2.csv', "/content/gdrive/MyDrive/1. Hasil Training CNN/Learning Rate Scheduler")

In [ ]:
# save model cnn
model.save('model_acc_100.h5')

# copy to google colab
!cp model_acc_100.h5 '/content/gdrive/MyDrive/1. Hasil Training CNN/Learning Rate Scheduler'

In [ ]:
# load image from google drive
input = image.load_img('/content/gdrive/MyDrive/Dataset Skripsi/Dataset Testing - Resized/18. Isi kurang, Tutup rusak, Label rusak/TEST_IK_TR_LR2.jpg', target_size=(SIZE,SIZE,3))

# preprocessing image
input = image.img_to_array(input)
input = input/255.
plt.imshow(input)
input = np.expand_dims(input, axis=0)

In [ ]:
# extract feature learning layer
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(input)
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
   # plt.figure(figsize = (20,20))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], aspect='equal')
            activation_index += 1

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# read csv from google colab
plotting = pd.read_csv('/content/gdrive/MyDrive/1. Hasil Training CNN/Learning Rate Scheduler/final graph_csv.csv')\

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = plotting['loss']
val_loss = plotting['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure(figsize=(13,8))
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()



acc = plotting['binary_accuracy']
val_acc = plotting['val_binary_accuracy']
plt.figure(figsize=(13,8))
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# save history training
hist_df = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'history_KFold_{}.csv'.format(i)
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# copy to drive
shutil.copy('history_KFold_{}.csv'.format(i), '/content/gdrive/MyDrive/1. Hasil Training CNN')


# copy to google colab
#!cp 'history_KFold_{}.csv'.format(i) '/content/gdrive/MyDrive/1. Hasil Training CNN'

In [ ]:
import shutil 

shutil.copy('History_KFold_{}.csv'.format(i), '/content/gdrive/MyDrive/1. Hasil Training CNN')

In [ ]:
model.save('Model_KFold{}.h5'.format(i))

shutil.copy('Model_KFold_{}.h5'.format(i), '/content/gdrive/MyDrive/1. Hasil Training CNN')

# copy to google colab
#!cp m '/content/gdrive/MyDrive/1. Hasil Training CNN'

In [ ]:
# open csv file
constant = pd.read_csv('/content/gdrive/MyDrive/1. Hasil Training CNN/hist_const_0_001.csv')
gstep = pd.read_csv('/content/gdrive/MyDrive/1. Hasil Training CNN/hist_sd_0_001.csv')
gtime = pd.read_csv('/content/gdrive/MyDrive/1. Hasil Training CNN/hist_tb_0_001.csv')
gext = pd.read_csv('/content/gdrive/MyDrive/1. Hasil Training CNN/hist_ed_0_001.csv')

In [ ]:
acc1 = gstep['binary_accuracy']
acc2 = gtime['binary_accuracy']
acc3 = gext['binary_accuracy']
acc4 = constant['binary_accuracy']

#val_acc = df['val_binary_accuracy']
plt.figure(figsize=(12,7))
plt.plot(epochs, acc4, 'g', label='Constant')
plt.plot(epochs, acc1, 'y', label='Step decay')
plt.plot(epochs, acc2, 'r', label='Time based')
plt.plot(epochs, acc3, 'b', label='Exponential decay')

#plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.legend()
plt.show()

In [ ]:
const = pd.read_csv('/content/gdrive/MyDrive/1. Hasil Training CNN/Copy of hist_lr_const.csv')

In [ ]:
lr1 = gstep['lr']
lr2 = gtime['lr']
lr3 = gext['lr']
lr4 = const['lr']

epochs = range(1, len(loss) + 1)
plt.figure(figsize=(12,7))
plt.plot(epochs, lr4, 'g', label='Constant')
plt.plot(epochs, lr1, 'y', label='Step decay')
plt.plot(epochs, lr2, 'r', label='Time based')
plt.plot(epochs, lr3, 'b', label='Exponential decay')

plt.title('Learning rate')
plt.xlabel('Epochs')
plt.ylabel('Learning rate')
plt.legend()
plt.show()

In [ ]:
loss1 = gstep['loss']
loss2 = gtime['loss']
loss3 = gext['loss']
loss4 = constant['loss']

#val_acc = df['val_binary_accuracy']
plt.figure(figsize=(12,7))
plt.plot(epochs, loss4, 'g', label='Constant')
plt.plot(epochs, loss1, 'y', label='Step decay')
plt.plot(epochs, loss2, 'r', label='Time based')
plt.plot(epochs, loss3, 'b', label='Exponential decay')

#plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.legend()
plt.show()

In [ ]:
%tensorboard --logdir logs/scalars

In [ ]:
!nvidia-smi -L

In [ ]:
!nvidia-smi

In [ ]:
!lscpu |grep 'Model name'

In [ ]:
!lscpu | grep 'Core(s) per socket:'

In [ ]:
!lscpu | grep 'Thread(s) per core'

In [ ]:
!free -h --si | awk  '/Mem:/{print $2}'

In [ ]:
!df -h / | awk '{print $4}'

In [ ]:
!cat /proc/cpuinfo

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
# cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, y):
  model = Sequential()

  model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(SIZE,SIZE,3)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Conv2D(filters=300, kernel_size=(5, 5), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(6, activation='sigmoid'))
 
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy" ,threshold=0.8)])
	# Fit the model
  model.fit(X[train], y=[train], epochs=100, batch_size=50, verbose=1)
	# evaluate the model
  scores = model.evaluate(X[test], y[test], verbose=1)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

In [ ]:
# import h5 model cnn file
from keras.models import load_model
model = load_model('/content/gdrive/MyDrive/1. Hasil Training CNN/KFold/Model_KFold_5.h5')

In [ ]:
# for test individualy and only show the present class label
img = image.load_img('IK_TR_LTA16.jpg', target_size=(SIZE,SIZE,3))

img = image.img_to_array(img)
img = img/255.
plt.imshow(img)
img = np.expand_dims(img, axis=0)

classes = np.array(df.columns[1:])
proba = model.predict(img)  

sorted_categories = np.argsort(proba[0])[:-7:-1]  

threshold = 0.8

for i in range(6): 
  if proba[0][sorted_categories[i]] > threshold :
    print("{}".format(classes[sorted_categories[i]])+" ({:.3})".format(proba[0][sorted_categories[i]]))

model.evaluate(XTest, YTest, verbose=1)

In [ ]:
# for test individually and show all class label
img = image.load_img('IMG_20201230_213044.jpg', target_size=(SIZE,SIZE,3))

img = image.img_to_array(img)
img = img/255.
plt.imshow(img)
img = np.expand_dims(img, axis=0)

classes = np.array(df.columns[1:])
proba = model.predict(img)  

sorted_categories = np.argsort(proba[0])[:-7:-1]  #Get class names for top 10 categories

#Print classes and corresponding probabilities
for i in range(6):
    print("{}".format(classes[sorted_categories[i]])+" ({:.3})".format(proba[0][sorted_categories[i]]))